## Import Library

In [218]:
#pip install pandas
import pandas as pd
import numpy as np
import string
import re
import datetime

## Variables

In [219]:
start_time = datetime.datetime.now()
target_user_id = 0
target_vote_number = 2000
numberOfResult = 5

## Loading Dataset
- Loading anime.csv, rating.csv and newRating.csv into DataFrame

In [220]:
anime_data = pd.read_csv('anime.csv')
anime_data = pd.DataFrame(anime_data)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [221]:
rating_data = pd.read_csv('rating.csv')
rating_data = pd.DataFrame(rating_data)
rating_data = rating_data[:5000000]

new_rating_data = pd.read_csv('newRating.csv')
rating_data = pd.DataFrame(rating_data)

rating_data = pd.concat([rating_data, new_rating_data])
rating_data

,user_id,anime_id,rating
0,1.0,20.0,-1.0
1,1.0,24.0,-1.0
2,1.0,79.0,-1.0
3,1.0,226.0,-1.0
4,1.0,241.0,-1.0
...,...,...,...
113,75009.0,18617.0,5.0
114,75009.0,25781.0,9.0
115,75009.0,5081.0,7.0
116,75009.0,29755.0,8.0


# Data Preprocessing

## Change variable type to appropraite data type

In [222]:
#anime file datatype
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [223]:
anime_data['episodes'] = anime_data['episodes'].replace('Unknown', -1)
anime_data = anime_data.astype({"name":"string","genre":"string","type":"string","episodes":"int64"})
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  12294 non-null  int64  
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), string(3)
memory usage: 672.5 KB


In [224]:
#rating file datatype
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   float64
 1   anime_id  float64
 2   rating    float64
dtypes: float64(3)
memory usage: 152.6 MB


In [225]:
rating_data = rating_data.replace([np.inf, -np.inf, np.nan], 0)
rating_data = rating_data.astype({"user_id":"int64", "anime_id":"int64", "rating":"int64"})
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 152.6 MB


## Rename column

In [226]:
anime_data = anime_data.rename(columns={"rating": "average_rating"})
anime_data.head(3)

,anime_id,name,genre,type,episodes,average_rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262


In [227]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head(3)

,user_id,anime_id,user_rating
0,1,20,-1
1,1,24,-1
2,1,79,-1


## Check Missing Value

In [228]:
rating_data.isna().sum()

user_id        0
anime_id       0
user_rating    0
dtype: int64

In [229]:
anime_data.isna().sum()

anime_id            0
name                0
genre              62
type               25
episodes            0
average_rating    230
members             0
dtype: int64

In [230]:
anime_data.dropna(axis=0, inplace= True)
anime_data.isna().sum()

anime_id          0
name              0
genre             0
type              0
episodes          0
average_rating    0
members           0
dtype: int64

## Check Duplicates

In [231]:
#check duplicate anime
duplicated_anime = anime_data[anime_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_anime}')

count of duplicated anime: 0


In [232]:
#check duplicate rating
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating: {duplicated_rating}')

count of duplicated rating: 2


In [233]:
rating_data.drop_duplicates(keep='first',inplace=True)
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating after removing: {duplicated_rating}')

count of duplicated rating after removing: 0


## Clean Text

In [234]:
def clean_text(text):
    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
  
    # remove punctuation
    #text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [235]:
#before clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [236]:
anime_data.loc[:, 'name'] = anime_data['name'].apply(clean_text)

In [237]:
#after clean text 28977 9969 15417
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


## Merge Dataset by anime_id

In [238]:
raw_merged_data = pd.merge(rating_data, anime_data, on='anime_id', sort = True)
raw_merged_data.sort_values(by='anime_id', ascending= False)

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
5000030,13954,34475,6,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
5000029,47765,34367,6,Long Riders Recap,"Comedy, Shounen, Slice of Life, Sports",Special,1,5.97,582
5000028,30565,34367,6,Long Riders Recap,"Comedy, Shounen, Slice of Life, Sports",Special,1,5.97,582
5000027,47421,34349,-1,Diamond no Ace: Second Season OVA,"Comedy, School, Shounen, Sports",OVA,2,7.25,1885
5000026,40821,34349,-1,Diamond no Ace: Second Season OVA,"Comedy, School, Shounen, Sports",OVA,2,7.25,1885
...,...,...,...,...,...,...,...,...,...
6272,29917,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
6271,29903,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
6270,29899,1,8,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
6269,29895,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824


## Drop Unnessary Row
Drop the row that less than rating number's mean value

In [239]:
#get number of rating for each anime
grouped_data = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count()).sort_values(by='user_rating', ascending= False)
grouped_data.describe()

,user_rating
count,9897.000000
mean,505.206729
std,1420.249029
min,1.000000
25%,7.000000
50%,45.000000
75%,305.000000
max,25637.000000


In [240]:
#get the anime that fulfilled the constaint which is more than mean value
trimmed_data = grouped_data[grouped_data['user_rating'] > grouped_data.mean()[0]]
#trim the data
raw_merged_data = raw_merged_data.loc[raw_merged_data['anime_id'].isin(trimmed_data.index.values.tolist())]
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
4999977,47536,34240,10,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
4999978,47678,34240,-1,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
4999979,47701,34240,-1,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
4999980,47702,34240,7,Shelter,"Music, Sci-Fi",Music,1,8.38,71136


# PART 1: User Based Correlation

## Read target user_id from user

In [241]:
target_user_id = int(input('Enter target user_id (75000 to 75009 is the new added user): '))
while target_user_id not in rating_data['user_id'].unique():
    target_user_id = int(input('Enter valid target user_id (75000 to 75009 is the new added user): '))

## Find rating_anime_matrix

In [242]:
rating_anime_matrix = raw_merged_data.pivot_table(index='user_id', columns=['anime_id'], values = 'user_rating', fill_value= -1)
rating_anime_matrix.head()

anime_id,1,5,6,7,15,16,18,19,20,22,...,32668,32681,32729,32828,32935,32998,33028,33558,34103,34240
user_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,8,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,8,-1,6,-1,6,-1,6,5,...,-1,-1,-1,7,-1,-1,-1,-1,-1,-1


## Find anime_rating_matrix

In [243]:
anime_rating_matrix = rating_anime_matrix.transpose()
anime_rating_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,75000,75001,75002,75003,75004,75005,75006,75007,75008,75009
anime_id,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
15,-1.0,-1.0,-1.0,-1.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Find the information of target_user_id

In [244]:
anime_rating_user = anime_rating_matrix[target_user_id]
pd.DataFrame(anime_rating_user.sort_values(ascending= False)).rename(columns={target_user_id: f"user_id {target_user_id}'s rating"})

,user_id 75001's rating
anime_id,
5114,9.0
11061,9.0
1535,9.0
136,8.0
14719,8.0
...,...
3228,-1.0
3226,-1.0
3225,-1.0


## Find Correlation between target_user_id with all the other users

In [245]:
similar_users = anime_rating_matrix.corrwith(anime_rating_user).dropna()
similar_users

user_id
1       -0.002746
2       -0.001372
3        0.098112
5        0.115442
7        0.058415
           ...   
75005   -0.004732
75006    0.140225
75007   -0.003863
75008    0.151476
75009    0.161843
Length: 45119, dtype: float64

## Sort and Format into Data Frame

In [246]:
sorted_similar_users = pd.DataFrame(similar_users, columns=['correlation']).sort_values(ascending= False, by= 'correlation')
sorted_similar_users

,correlation
user_id,
75001,1.000000
47354,0.740115
9639,0.620118
29006,0.565519
43533,0.565519
...,...
19593,-0.039100
10307,-0.039976
22987,-0.042602


## Find the best correlated user

In [247]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]
most_similar_user

47354

## Find 5 best rated anime from the user that have highest correlation value

In [248]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]

#sort value by rating
user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)

#Find target_user's rated anime
target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)

#Trim the result exist in user's rating
user_recommendation = user_recommendation[~user_recommendation['anime_id'].isin(target_user['anime_id'].tolist())]

In [249]:
#validate the number of recommendation fulfill requirement, if not, move to the second highest correlation user
n = 1
while(len(user_recommendation) < 5):
    n = n + 1
    #most similar user is the user that at index number n
    most_similar_user = sorted_similar_users.index.values
    most_similar_user = most_similar_user[n]

    #sort value by rating
    user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
    user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)

    #Find target_user's rated anime
    target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)

    #Trim the result exist in user's rating
    user_recommendation = user_recommendation[~user_recommendation['anime_id'].isin(target_user['anime_id'].tolist())]
    
user_recommendation = user_recommendation[:numberOfResult]
user_recommendation

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
475929,38352,269,10,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055
248493,38352,137,9,Hunter x Hunter OVA,"Action, Adventure, Shounen, Super Power",OVA,8,8.41,53168
42643,38352,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
250219,38352,138,-1,Hunter x Hunter: Greed Island,"Action, Adventure, Shounen, Super Power",OVA,8,8.33,57029
251907,38352,139,-1,Hunter x Hunter: Greed Island Final,"Action, Adventure, Shounen, Super Power",OVA,14,8.41,55787


In [250]:
user_recommendation_list = user_recommendation['anime_id'].tolist()
user_recommendation_list

[269, 137, 20, 138, 139]

# PART 2: Item Based Correlation

## Use the user's highest rated anime to recommend other related anime

In [251]:
source_user_anime_rating = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)
pd.DataFrame(source_user_anime_rating.head(10))

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
1216243,75001,1535,9,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
2110445,75001,5114,9,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
3249394,75001,11061,9,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
247189,75001,136,8,Hunter x Hunter,"Action, Adventure, Shounen, Super Power",TV,62,8.48,166255
3721646,75001,14719,8,JoJo no Kimyou na Bouken (TV),"Action, Adventure, Shounen, Supernatural, Vampire",TV,26,8.51,190197
4301583,75001,20899,8,JoJo no Kimyou na Bouken: Stardust Crusaders,"Action, Adventure, Drama, Shounen, Supernatural",TV,24,8.25,120089
4648317,75001,26055,8,JoJo no Kimyou na Bouken: Stardust Crusaders 2...,"Action, Adventure, Drama, Shounen, Supernatural",TV,24,8.60,93657


In [252]:
top_rated_anime_id = source_user_anime_rating['anime_id'].tolist()[0]
top_rated_anime_id

1535

In [253]:
ratings = rating_anime_matrix[top_rated_anime_id]
pd.DataFrame(ratings).rename(columns={top_rated_anime_id: f"{top_rated_anime_id}'s rating"})

,1535's rating
user_id,
1,-1
2,-1
3,10
4,-1
5,4
...,...
75005,-1
75006,-1
75007,-1


In [254]:
similar_animes = rating_anime_matrix.corrwith(ratings)
similar_animes = pd.DataFrame(similar_animes, columns=['correlation'])
similar_animes

,correlation
anime_id,
1,0.186228
5,0.111478
6,0.163428
7,0.062119
15,0.093207
...,...
32998,0.093597
33028,0.060910
33558,0.043396


## Identify the most correlated anime

In [255]:
sorted_similar_anime = pd.DataFrame(similar_animes, columns=['correlation']).sort_values(by= 'correlation', ascending= False)
sorted_similar_anime

,correlation
anime_id,
1535,1.000000
1575,0.389517
2904,0.372178
226,0.340161
5114,0.332886
...,...
9441,0.012674
1110,0.011179
6586,0.009679


## Eliminate the source anime

In [256]:
sorted_similar_anime = sorted_similar_anime[1:]
sorted_similar_anime

,correlation
anime_id,
1575,0.389517
2904,0.372178
226,0.340161
5114,0.332886
16498,0.316169
...,...
9441,0.012674
1110,0.011179
6586,0.009679


## Ensure the identified anime is popular
Higher number of votes means more popular

In [257]:
#Get number of rating for each anime
rating_votes = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
rating_votes=rating_votes.rename(columns={'user_rating': 'rating_count'})

In [258]:
similar_animes_ratings = sorted_similar_anime.join(rating_votes['rating_count']).sort_values(by = 'correlation', ascending = False)
similar_animes_ratings

,correlation,rating_count
anime_id,,
1575,0.389517,17718
2904,0.372178,15663
226,0.340161,17550
5114,0.332886,16825
16498,0.316169,20603
...,...,...
9441,0.012674,600
1110,0.011179,521
6586,0.009679,814


## Get the anime that have higher votes and have higher correlation

In [259]:
similar_popular_animes = similar_animes_ratings.loc[similar_animes_ratings['rating_count']>=target_vote_number].dropna()
similar_popular_animes

,correlation,rating_count
anime_id,,
1575,0.389517,17718
2904,0.372178,15663
226,0.340161,17550
5114,0.332886,16825
16498,0.316169,20603
...,...,...
8557,0.044788,2269
14811,0.043498,2030
12403,0.041899,2246


In [260]:
#trim the result exist in user's rating
similar_popular_animes = similar_popular_animes[~similar_popular_animes.index.isin(target_user['anime_id'].tolist())]
similar_popular_animes

,correlation,rating_count
anime_id,,
1575,0.389517,17718
2904,0.372178,15663
226,0.340161,17550
16498,0.316169,20603
20,0.310089,16725
...,...,...
8557,0.044788,2269
14811,0.043498,2030
12403,0.041899,2246


In [261]:
#make it a list
most_similar_popular_animes = similar_popular_animes[:numberOfResult]
most_similar_popular_animes_list = most_similar_popular_animes.index.to_list()
most_similar_popular_animes_list

[1575, 2904, 226, 16498, 20]

# Result Showing

In [262]:
#combine the user-based and item-based result
final_recommendation_result = user_recommendation_list.copy()
final_recommendation_result += most_similar_popular_animes_list.copy()
final_recommendation_result

[269, 137, 20, 138, 139, 1575, 2904, 226, 16498, 20]

## Show Source User's Rating

In [263]:
pd.DataFrame(source_user_anime_rating)

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
1216243,75001,1535,9,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
2110445,75001,5114,9,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
3249394,75001,11061,9,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
247189,75001,136,8,Hunter x Hunter,"Action, Adventure, Shounen, Super Power",TV,62,8.48,166255
3721646,75001,14719,8,JoJo no Kimyou na Bouken (TV),"Action, Adventure, Shounen, Supernatural, Vampire",TV,26,8.51,190197
4301583,75001,20899,8,JoJo no Kimyou na Bouken: Stardust Crusaders,"Action, Adventure, Drama, Shounen, Supernatural",TV,24,8.25,120089
4648317,75001,26055,8,JoJo no Kimyou na Bouken: Stardust Crusaders 2...,"Action, Adventure, Drama, Shounen, Supernatural",TV,24,8.60,93657


## Show User Correlation Table

In [264]:
sorted_similar_users

,correlation
user_id,
75001,1.000000
47354,0.740115
9639,0.620118
29006,0.565519
43533,0.565519
...,...
19593,-0.039100
10307,-0.039976
22987,-0.042602


## Show Most Similar User's Rating
- The user might not be the most correlated if the number of rated anime is less than 5

In [265]:
most_similar_user_df = raw_merged_data.loc[raw_merged_data['user_id'] == most_similar_user].sort_values(by= 'user_rating', ascending= False)
most_similar_user_df.head(10)
user_recommendation

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
475929,38352,269,10,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055
248493,38352,137,9,Hunter x Hunter OVA,"Action, Adventure, Shounen, Super Power",OVA,8,8.41,53168
42643,38352,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
250219,38352,138,-1,Hunter x Hunter: Greed Island,"Action, Adventure, Shounen, Super Power",OVA,8,8.33,57029
251907,38352,139,-1,Hunter x Hunter: Greed Island Final,"Action, Adventure, Shounen, Super Power",OVA,14,8.41,55787


## Show Target Anime's information 
- Anime is picked from the source user's top rating

In [266]:
target_search_anime = anime_data.loc[anime_data['anime_id'] == top_rated_anime_id]
pd.DataFrame(target_search_anime)

,anime_id,name,genre,type,episodes,average_rating,members
40,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917


## Show Anime Correlation Table

In [267]:
similar_popular_animes

,correlation,rating_count
anime_id,,
1575,0.389517,17718
2904,0.372178,15663
226,0.340161,17550
16498,0.316169,20603
20,0.310089,16725
...,...,...
8557,0.044788,2269
14811,0.043498,2030
12403,0.041899,2246


In [268]:
# Item-based recommended result
most_similar_popular_animes_df = anime_data.loc[anime_data['anime_id'].isin(most_similar_popular_animes_list)]
most_similar_popular_animes_df

,anime_id,name,genre,type,episodes,average_rating,members
13,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888
19,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
760,226,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
841,20,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


## Show Final Recommendation Result
- The recommendation might not enought 10 if there is duplicated anime

In [269]:
#Drop the duplicated anime
recommended_anime = anime_data.loc[anime_data['anime_id'].isin(final_recommendation_result)]
recommended_anime.drop_duplicates(subset=['anime_id'], keep='first')

pd.DataFrame(recommended_anime)

,anime_id,name,genre,type,episodes,average_rating,members
13,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888
19,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
145,137,Hunter x Hunter OVA,"Action, Adventure, Shounen, Super Power",OVA,8,8.41,53168
146,139,Hunter x Hunter: Greed Island Final,"Action, Adventure, Shounen, Super Power",OVA,14,8.41,55787
202,138,Hunter x Hunter: Greed Island,"Action, Adventure, Shounen, Super Power",OVA,8,8.33,57029
582,269,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055
760,226,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
841,20,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


In [270]:
end_time = datetime.datetime.now()
# Calculate the time elapsed
elapsed_time = end_time - start_time

# Print the elapsed time in seconds
print("Elapsed time in seconds:", elapsed_time.total_seconds())

Elapsed time in seconds: 48.844725
